# FireProx Phase 2 Feature Demo

This notebook demonstrates the new Phase 2 features:
- **Field-Level Dirty Tracking** - Fine-grained change detection
- **Partial Updates** - Efficient updates sending only modified fields
- **Subcollections** - Hierarchical data structures
- **Atomic Operations** - ArrayUnion, ArrayRemove, Increment

The demo is split into two sections:
1. Synchronous API examples
2. Asynchronous API examples

## Setup

Import the necessary modules for both sync and async APIs.

In [19]:
from fire_prox import FireProx, AsyncFireProx
from fire_prox.testing import demo_client, async_demo_client

---

# Part 1: Synchronous API Examples

The following examples use the synchronous FireProx API.

### Initialize Sync Client

In [20]:
# Create sync client and collection
client = demo_client()
db = FireProx(client)
users = db.collection('phase2_demo_users')

## Feature 1: Field-Level Dirty Tracking

Track exactly which fields have been modified since the last save.

In [4]:
# Create a user with multiple fields
user = users.new()
user.name = 'Ada Lovelace'
user.year = 1815
user.occupation = 'Mathematician'
user.country = 'England'
user.save(doc_id='ada_sync')

print(f"Initial state: dirty={user.is_dirty()}")

# Modify only some fields
user.year = 1816
user.occupation = 'Computer Pioneer'

# Inspect which fields changed
print(f"\nAfter changes:")
print(f"  is_dirty: {user.is_dirty()}")
print(f"  dirty_fields: {user.dirty_fields}")
print(f"  deleted_fields: {user.deleted_fields}")

# Save changes
user.save()
print(f"\nAfter save: dirty={user.is_dirty()}")

Initial state: dirty=False

After changes:
  is_dirty: True
  dirty_fields: {'year', 'occupation'}
  deleted_fields: set()

After save: dirty=False


## Feature 2: Partial Updates

Only modified fields are sent to Firestore, reducing bandwidth and costs.

In [5]:
# Create a document with many fields
user = users.new()
user.name = 'Charles Babbage'
user.year = 1791
user.occupation = 'Mathematician'
user.country = 'England'
user.field1 = 'data1'
user.field2 = 'data2'
user.field3 = 'data3'
user.save(doc_id='charles_sync')

print("Document created with 8 fields")

# Modify only ONE field
user.occupation = 'Inventor'
print(f"\nModified 1 field out of 8")
print(f"  dirty_fields: {user.dirty_fields}")

# Save - only sends the modified field!
user.save()
print("\nSave complete - only 1 field sent to Firestore (87.5% reduction!)")

# Delete a field
del user.field3
print(f"\nDeleted field3")
print(f"  deleted_fields: {user.deleted_fields}")

user.save()
print("Field deletion saved to Firestore")

Document created with 8 fields

Modified 1 field out of 8
  dirty_fields: {'occupation'}

Save complete - only 1 field sent to Firestore (87.5% reduction!)

Deleted field3
  deleted_fields: {'field3'}
Field deletion saved to Firestore


## Feature 3: Subcollections

Create hierarchical data structures with documents inside documents.

In [6]:
# Create a parent document (user)
user = users.new()
user.name = 'Grace Hopper'
user.year = 1906
user.save(doc_id='grace_sync')

print(f"Created user: {user.path}")

# Access subcollection
posts = user.collection('posts')
print(f"\nAccessed subcollection: {posts.id}")

# Create documents in the subcollection
post1 = posts.new()
post1.title = 'The First Compiler'
post1.year = 1952
post1.save(doc_id='compiler')
print(f"\nCreated post: {post1.path}")

post2 = posts.new()
post2.title = 'COBOL Development'
post2.year = 1959
post2.save(doc_id='cobol')
print(f"Created post: {post2.path}")

# Nested subcollections (3 levels deep!)
comments = post1.collection('comments')
comment = comments.new()
comment.text = 'Revolutionary work!'
comment.author = 'Anonymous'
comment.save(doc_id='comment1')
print(f"\nCreated nested comment: {comment.path}")

Created user: phase2_demo_users/grace_sync

Accessed subcollection: posts

Created post: phase2_demo_users/grace_sync/posts/compiler
Created post: phase2_demo_users/grace_sync/posts/cobol

Created nested comment: phase2_demo_users/grace_sync/posts/compiler/comments/comment1


## Feature 4: Atomic Operations

Perform array and counter operations without reading the document first.

### Array Union - Add elements to arrays

In [7]:
# Create a user with tags
user = users.new()
user.name = 'Alan Turing'
user.tags = ['mathematics', 'cryptography']
user.save(doc_id='alan_sync')

print(f"Initial tags: {user.tags}")

# Add more tags using array_union (no read required!)
user.array_union('tags', ['computer-science', 'ai'])
user.save()

# Fetch to see updated tags
user.fetch(force=True)
print(f"\nAfter array_union: {sorted(user.tags)}")

# array_union automatically deduplicates
user.array_union('tags', ['ai', 'biology'])  # 'ai' already exists
user.save()

user.fetch(force=True)
print(f"After duplicate add: {sorted(user.tags)}")
print("Note: 'ai' wasn't added twice (automatic deduplication)")

Initial tags: ['mathematics', 'cryptography']

After array_union: ['ai', 'computer-science', 'cryptography', 'mathematics']
After duplicate add: ['ai', 'biology', 'computer-science', 'cryptography', 'mathematics']
Note: 'ai' wasn't added twice (automatic deduplication)


### Array Remove - Remove elements from arrays

In [8]:
# Remove tags using array_remove
user.array_remove('tags', ['biology'])
user.save()

user.fetch(force=True)
print(f"After array_remove: {sorted(user.tags)}")

After array_remove: ['ai', 'computer-science', 'cryptography', 'mathematics']


### Increment - Atomic counter operations

In [9]:
# Create a blog post with counters
posts_collection = db.collection('phase2_demo_posts')
post = posts_collection.new()
post.title = 'Understanding Atomic Operations'
post.views = 100
post.likes = 10
post.save(doc_id='post1_sync')

print(f"Initial: views={post.views}, likes={post.likes}")

# Increment view counter (concurrency-safe!)
post.increment('views', 1)
post.save()

post.fetch(force=True)
print(f"\nAfter increment: views={post.views}")

# Decrement with negative value
post.increment('likes', -2)
post.save()

post.fetch(force=True)
print(f"After decrement: likes={post.likes}")

# Multiple operations at once!
post.increment('views', 5)
post.increment('likes', 3)
post.save()

post.fetch(force=True)
print(f"\nAfter multiple ops: views={post.views}, likes={post.likes}")

Initial: views=100, likes=10

After increment: views=101
After decrement: likes=8

After multiple ops: views=106, likes=11


### Combined Operations

Mix atomic operations with regular field updates in a single save.

In [10]:
# Combine everything in one save!
user = users.doc('alan_sync')
user.fetch()

print(f"Before: name={user.name}, tags={sorted(user.tags)}")

# Regular field update
user.status = 'legendary'

# Atomic array operation
user.array_union('tags', ['turing-machine'])

# All applied atomically in one save!
user.save()

user.fetch(force=True)
print(f"\nAfter combined ops:")
print(f"  status: {user.status}")
print(f"  tags: {sorted(user.tags)}")

Before: name=Alan Turing, tags=['ai', 'computer-science', 'cryptography', 'mathematics']

After combined ops:
  status: legendary
  tags: ['ai', 'computer-science', 'cryptography', 'mathematics', 'turing-machine']


---

# Part 2: Asynchronous API Examples

The following examples use the asynchronous AsyncFireProx API with async/await.

### Initialize Async Client

In [11]:
# Create async client and collection
async_client = async_demo_client()
async_db = AsyncFireProx(async_client)
async_users = async_db.collection('phase2_demo_users_async')

## Feature 1: Field-Level Dirty Tracking (Async)

In [12]:
# Create a user with multiple fields
user = async_users.new()
user.name = 'Ada Lovelace'
user.year = 1815
user.occupation = 'Mathematician'
user.country = 'England'
await user.save(doc_id='ada_async')

print(f"Initial state: dirty={user.is_dirty()}")

# Modify only some fields
user.year = 1816
user.occupation = 'Computer Pioneer'

# Inspect which fields changed
print(f"\nAfter changes:")
print(f"  is_dirty: {user.is_dirty()}")
print(f"  dirty_fields: {user.dirty_fields}")
print(f"  deleted_fields: {user.deleted_fields}")

# Save changes
await user.save()
print(f"\nAfter save: dirty={user.is_dirty()}")

Initial state: dirty=False

After changes:
  is_dirty: True
  dirty_fields: {'year', 'occupation'}
  deleted_fields: set()

After save: dirty=False


## Feature 2: Partial Updates (Async)

In [13]:
# Create a document with many fields
user = async_users.new()
user.name = 'Charles Babbage'
user.year = 1791
user.occupation = 'Mathematician'
user.country = 'England'
user.field1 = 'data1'
user.field2 = 'data2'
user.field3 = 'data3'
await user.save(doc_id='charles_async')

print("Document created with 8 fields")

# Modify only ONE field
user.occupation = 'Inventor'
print(f"\nModified 1 field out of 8")
print(f"  dirty_fields: {user.dirty_fields}")

# Save - only sends the modified field!
await user.save()
print("\nSave complete - only 1 field sent to Firestore (87.5% reduction!)")

# Delete a field
del user.field3
print(f"\nDeleted field3")
print(f"  deleted_fields: {user.deleted_fields}")

await user.save()
print("Field deletion saved to Firestore")

Document created with 8 fields

Modified 1 field out of 8
  dirty_fields: {'occupation'}

Save complete - only 1 field sent to Firestore (87.5% reduction!)

Deleted field3
  deleted_fields: {'field3'}
Field deletion saved to Firestore


## Feature 3: Subcollections (Async)

In [14]:
# Create a parent document (user)
user = async_users.new()
user.name = 'Grace Hopper'
user.year = 1906
await user.save(doc_id='grace_async')

print(f"Created user: {user.path}")

# Access subcollection
posts = user.collection('posts')
print(f"\nAccessed subcollection: {posts.id}")

# Create documents in the subcollection
post1 = posts.new()
post1.title = 'The First Compiler'
post1.year = 1952
await post1.save(doc_id='compiler')
print(f"\nCreated post: {post1.path}")

post2 = posts.new()
post2.title = 'COBOL Development'
post2.year = 1959
await post2.save(doc_id='cobol')
print(f"Created post: {post2.path}")

# Nested subcollections (3 levels deep!)
comments = post1.collection('comments')
comment = comments.new()
comment.text = 'Revolutionary work!'
comment.author = 'Anonymous'
await comment.save(doc_id='comment1')
print(f"\nCreated nested comment: {comment.path}")

Created user: phase2_demo_users_async/grace_async

Accessed subcollection: posts

Created post: phase2_demo_users_async/grace_async/posts/compiler
Created post: phase2_demo_users_async/grace_async/posts/cobol

Created nested comment: phase2_demo_users_async/grace_async/posts/compiler/comments/comment1


## Feature 4: Atomic Operations (Async)

### Array Union (Async)

In [15]:
# Create a user with tags
user = async_users.new()
user.name = 'Alan Turing'
user.tags = ['mathematics', 'cryptography']
await user.save(doc_id='alan_async')

print(f"Initial tags: {user.tags}")

# Add more tags using array_union
user.array_union('tags', ['computer-science', 'ai'])
await user.save()

# Fetch to see updated tags
await user.fetch(force=True)
print(f"\nAfter array_union: {sorted(user.tags)}")

# array_union automatically deduplicates
user.array_union('tags', ['ai', 'biology'])
await user.save()

await user.fetch(force=True)
print(f"After duplicate add: {sorted(user.tags)}")
print("Note: 'ai' wasn't added twice (automatic deduplication)")

Initial tags: ['mathematics', 'cryptography']

After array_union: ['ai', 'computer-science', 'cryptography', 'mathematics']
After duplicate add: ['ai', 'biology', 'computer-science', 'cryptography', 'mathematics']
Note: 'ai' wasn't added twice (automatic deduplication)


### Array Remove (Async)

In [16]:
# Remove tags using array_remove
user.array_remove('tags', ['biology'])
await user.save()

await user.fetch(force=True)
print(f"After array_remove: {sorted(user.tags)}")

After array_remove: ['ai', 'computer-science', 'cryptography', 'mathematics']


### Increment (Async)

In [17]:
# Create a blog post with counters
async_posts = async_db.collection('phase2_demo_posts_async')
post = async_posts.new()
post.title = 'Understanding Atomic Operations'
post.views = 100
post.likes = 10
await post.save(doc_id='post1_async')

print(f"Initial: views={post.views}, likes={post.likes}")

# Increment view counter
post.increment('views', 1)
await post.save()

await post.fetch(force=True)
print(f"\nAfter increment: views={post.views}")

# Decrement with negative value
post.increment('likes', -2)
await post.save()

await post.fetch(force=True)
print(f"After decrement: likes={post.likes}")

# Multiple operations at once!
post.increment('views', 5)
post.increment('likes', 3)
await post.save()

await post.fetch(force=True)
print(f"\nAfter multiple ops: views={post.views}, likes={post.likes}")

Initial: views=100, likes=10

After increment: views=101
After decrement: likes=8

After multiple ops: views=106, likes=11


### Combined Operations (Async)

In [18]:
# Combine everything in one save!
user = async_users.doc('alan_async')
await user.fetch()

print(f"Before: name={user.name}, tags={sorted(user.tags)}")

# Regular field update
user.status = 'legendary'

# Atomic array operation
user.array_union('tags', ['turing-machine'])

# All applied atomically in one save!
await user.save()

await user.fetch(force=True)
print(f"\nAfter combined ops:")
print(f"  status: {user.status}")
print(f"  tags: {sorted(user.tags)}")

Before: name=Alan Turing, tags=['ai', 'computer-science', 'cryptography', 'mathematics']

After combined ops:
  status: legendary
  tags: ['ai', 'computer-science', 'cryptography', 'mathematics', 'turing-machine']


---

## Summary

This demo showcased all Phase 2 features:

✅ **Field-Level Dirty Tracking** - Inspect exactly what changed with `.dirty_fields` and `.deleted_fields`

✅ **Partial Updates** - 50-90% bandwidth reduction by sending only modified fields

✅ **Subcollections** - Hierarchical data with `.collection()` method, unlimited nesting

✅ **Atomic Operations**:
  - `array_union()` - Add elements to arrays (with auto-deduplication)
  - `array_remove()` - Remove elements from arrays
  - `increment()` - Atomic counter operations (concurrency-safe)

All features work identically in both sync and async APIs!

### Performance Benefits

- **Bandwidth**: 50-90% reduction from partial updates
- **Concurrency**: Atomic operations prevent race conditions
- **Cost**: Lower Firestore costs from reduced data transfer
- **Speed**: Fewer network round-trips for counters and arrays

### Learn More

See `docs/PHASE2_IMPLEMENTATION_REPORT.md` for complete documentation.